In [7]:
import pandas as pd
import pickle
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

def imputate_data(df):
    df.dropna(axis=1, how='all', inplace=True)

    df1 = df.filter(regex='id')
    df1.fillna(df1.mode().iloc[0], axis=0, inplace=True)

    df.fillna(df.mean(), axis=0, inplace=True)
    df[list(df1.keys())] = df1

def encode_objects(df):
    for f in df.columns:
        if df[f].dtype=='object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df[f].values))
            df[f] = lbl.transform(list(df[f].values))

def split_merged(df):
    Y = df.logerror.values
    X = df.drop(["parcelid", "transactiondate", "logerror"], axis=1)
    return X, Y

In [9]:
print("Loading data...")
transactions = pd.read_csv('data/train_2016_v2.csv', parse_dates=['transactiondate'])
test_transactions = pd.read_csv('data/train_2017.csv', parse_dates=['transactiondate'])
properties = pd.read_csv('data/properties_2016.csv')
print("Done.")
print("-------------------------------------------------------------------")

print("Parsing data...")
merged = pd.merge(transactions, properties, on="parcelid", how="left")
test_merged = pd.merge(test_transactions, properties, on="parcelid", how="left")

imputate_data(merged)
imputate_data(test_merged)

encode_objects(merged)
encode_objects(test_merged)

X, Y = split_merged(merged)
X_test, Y_test = split_merged(test_merged)

split_index = round(len(X)*0.7) # 70-30 training/validation split
X_train, Y_train, X_validate, Y_validate = (X[:split_index], Y[:split_index],
                                            X[split_index:], Y[split_index:])

Loading data...


/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Done.
-------------------------------------------------------------------
Parsing data...


In [18]:
model = pickle.load(open("model.p", "rb"))
print(model.score(X_train, Y_train))
print(mean_squared_error(Y_test, model.predict(X_test)))

print(model.feature_importances_)

0.10572490719
0.0333701327718
